In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
#default_exp callbacks

# Callbacks

Different Callbacks to be called during Learning Process

In [5]:
#export
#fastai callbacks
from PrimeCNNv3.imports import *
from PrimeCNNv3.utils.vizualize import plot_loss_update
from PrimeCNNv3.metric import accuracy

In [6]:
#export
class Callbacks(GetAttr): _default='learner'

In [56]:
#export
class SetupLearnerCB(Callbacks):
    def before_fit(self):
        self.learner.mb = master_bar(range(self.epochs))
        self.model.to(self.device)        
        
    def before_batch(self):
    
        xb, yb = self.batch
        self.learner.xb = xb.to(self.device, non_blocking = True)
        self.learner.yb = yb.to(self.device, non_blocking = True)
    
    def before_train_epoch(self):
        self.model.train()
        self.learner.training = True
    
    def before_valid_epoch(self):
        self.model.eval()
        self.learner.training = False
   
    def after_validation(self):
        pass
      #  self.mb.write(f'Epoch: {self.epoch}, Training Loss: {self.epoch_loss.train[-1]}, validation Loss: {self.epoch_loss.valid[-1]}, Training Accuracy: {self.epoch_acc.train[-1]}, Validation Accuracy: {self.epoch_acc.valid[-1]}')
      #  plot_loss_update(self.epoch, self.epochs,self.mb, train_loss=self.epoch_loss.train,valid_loss=self.epoch_loss.valid)

In [153]:
#export
class Recorder(Callbacks):
    '''Records loss and lrs'''
    def __init__(self):
        self.loss = TrackTrainVal()
        
        
        #track loss across an epoch
        self.track_loss = AvgLosses()
        
        self.track_train_smoothLoss = AvgSmoothLoss()
        self.epoch_metricTracker = TrackTrainVal()
        
        self.accumetric = AccumMetric()
        
       
        
    def before_fit(self):
        self.learner.losses = self.loss
        self.losses.reset()
        self.track_loss.reset()
        self.track_train_smoothLoss.reset()
        self.epoch_metricTracker.reset()
        
        self.learner.lrs = []
        self.learner.train_iter = 0
   
    def before_epoch(self):
        self.accumetric.reset()
        self.start_epoch = time.time()
    
    def after_epoch(self):
        self.learner.log = [self.epoch, round(self.losses.train[-1],6), round(self.losses.valid[-1],6)]
      
        for met in self.metric:
            m = round(met(preds = self.accumetric.value[0], target = self.accumetric.value[1]),6)
            self.learner.log.append(m)
            
        self.learner.log.append(format_time(time.time() - self.start_epoch))
        
    
    def before_valid_epoch(self):
        self.track_loss.reset()
        
    
    def after_train_epoch(self):
        #self.learner.losses.train.append(self.track_train_smoothLoss.value)
        pass
    
    def after_valid_epoch(self):
        self.losses.valid.append(self.track_loss.value)
        
    def after_batch(self):
        if self.training:
            
            self.learner.train_iter += 1
            #send leaner to track_loss 
            
            #track learning rate
            self.learner.lrs.append(self.opt.param_groups[0]['lr'])
            
            #track smoothloss for plotting
            self.track_train_smoothLoss.value = self.learner
            self.losses.train.append(self.track_train_smoothLoss.value)
            
            
            
        else:
            self.track_loss.value = self.learner
            self.accumetric.value = self.learner
            
            

In [154]:
#export
class ShowStats(Callbacks):
    def before_fit(self):
        
        self.header_line =  ['Epoch', 'train_loss', 'valid_loss','accuracy','precision','recall','f1_score', 'time']
        # loop over metric and add function name to header_line
        self.bs = []
        self.learner.log = []
        self.mb.write(self.header_line, table = True)
    def after_train_epoch(self):
        self.bs.append(self.train_iter)
        
    def after_epoch(self):
        plot_loss_update(nbs_batches = self.bs, epochs = self.epochs, mb = self.mb, 
                         train_loss = self.losses.train, valid_loss = self.losses.valid)
        
        self.mb.write(self.log, table = True)
        

In [155]:
#export
class TrackTrainVal:
    '''Wrapper to store train/valid stats'''
    def __init__(self):
        self.train, self.valid = [],[]

    def reset(self):
        self.train.clear()
        self.valid.clear()

In [156]:
#export
class AvgMetric:
    '''Average metric tracking'''
    def __init__(self, func):
        self.func = func
        self.bs_counter = 0
        self.total = 0.
    
    @property
    def value(self):
        return self.total / self.bs_counter
    
    @value.setter
    def value(self, learn):
        self.total += self.func(learn.preds, learn.yb)
        self.bs_counter += learn.batch_size
    
    def reset(self):
        self.bs_counter = 0
        self.total = 0.

In [157]:
#export
class AccumMetric:
    '''
        Accumulate prediction and target
        apply func to get the metric
    '''
    def __init__(self):
        
        self.hist_preds = []
        self.hist_target = []
        self.preds = []
        self.target = []
        
    @property
    def value(self):
        return torch.cat(self.preds), torch.cat(self.target)
    
    @value.setter
    def value(self, learn):
        self.preds.append(learn.preds.cpu().detach())
        self.target.append(learn.yb.cpu().detach())
       
        
    def reset(self):
        self.hist_preds.extend(self.preds)
        self.hist_target.extend(self.target)
        self.preds.clear()
        self.target.clear()
        
        

In [158]:
#export
class AvgLosses:
    '''keep track of losses'''
    #used to print loss stats after each epoch
    def __init__(self):
        self.bs_counter = 0
        self.total = 0.
        
    @property
    def value(self):
        return self.total / self.bs_counter
    
    @value.setter
    def value(self, learn):
        self.total += learn.running_loss * learn.batch_size
        self.bs_counter += learn.batch_size
        
    def reset(self):
        self.bs_counter = 0
        self.total = 0.

class AvgSmoothLoss:
    ''' exponentially weigth loss'''
    #mainly used for plotting the loss grpah w.r.t iteration
    def __init__(self, beta = 0.98):
        '''
            Args:
                beta: for exponentially weighted average
        '''
        self.smoothloss = torch.tensor(0)
        self.counter = 0
        self.beta = beta
        
    @property
    def value(self):
        return self.smoothloss / (1 - self.beta**self.counter)
    
    @value.setter
    def value(self, learn):
        self.counter += 1
        #self.smoothloss = (self.smoothloss * self.beta) + ((1 - self.beta) * learn.running_loss)
        #lerp
        self.smoothloss = learn.running_loss + self.beta * (self.smoothloss - learn.running_loss)
        
    def reset(self, beta = 0.98):
        self.counter = 0
        self.smoothloss = 0
        self.beta = beta
                                                               
        

In [159]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_utils.data.dataset.ipynb.
Converted 01_utils.data.dataloaders.ipynb.
Converted 02_utils.vizualize.ipynb.
Converted 03_callbacks.ipynb.
Converted 04_learner.ipynb.
Converted 05_metrics.ipynb.
Converted index.ipynb.
